In [85]:
# merge all the files with the affix ".xlsx" in the current folder into one file

import pandas as pd
import os


In [114]:
def aggregate_abstracts(FILE_NAME):
    # read the merged dataframe
    df = pd.read_excel(FILE_NAME)
    # print the number of rows in the merged dataframe
    print(len(df))
    
    # print(df.columns)
    # print(df.head())
    if len(df.columns) == 9:
        df.columns = ['article_id', 'abstract', 'Background', 'Purpose', 'Method', 'Result', 'Conclusions', 'Limitation', 'Others']
    if len(df.columns) == 14:
        df.columns = ['dataset_type', 'row_id', 'article_id', 'category', 'abstract', 'Background', 'Purpose', 'Method', 'Result', 'Conclusions', 'Limitation', 'Others',  'Errors', 'TooLong']
        assert len(df.columns) == 14

    
    abstracts = {}
    facets_for_abstracts = {}
    for row in df.iterrows():
        # split the abstract into sentences according to the tages <S0>, <S1>, <S2>, ...
        sentences =  row[1]['abstract'].split("<S")[1:]        
        sentences = [sentence[sentence.find(">")+1:] for sentence in sentences] # get the content of each sentence
        sentences = [sentence[:sentence.find("</S")] for sentence in sentences] # remove the tags </S0>, </S1>, </S2>, ...

        # extract facet for each sentence
        facets = [None] * len(sentences)
        for i in range(len(sentences)):
            identifier = 'S'+str(i)
            for tag in [ 'Background', 'Purpose', 'Method', 'Result', 'Conclusions', 'Limitation', 'Others']:
                sent_ids = row[1][tag]
                if type(sent_ids) == float:
                    continue
                if identifier in sent_ids:
                    if facets[i] == None:
                        facets[i] = [tag]
                    else:
                        facets[i].append(tag)
            if facets[i] == None:
                facets[i] = ["Others"]
                    

        abstracts[str(row[1]['article_id'])] = sentences
        facets_for_abstracts[str(row[1]['article_id'])] = facets
    return abstracts, facets_for_abstracts


abstracts, facets_for_abstracts = aggregate_abstracts("test.xlsx")

40


In [116]:
facets_for_abstracts

{'1801.07358': [['Background', 'Purpose'],
  ['Method'],
  ['Result', 'Conclusions']],
 '1108.2937': [['Background'],
  ['Background'],
  ['Limitation'],
  ['Purpose', 'Method'],
  ['Method'],
  ['Result', 'Conclusions'],
  ['Others']],
 '1502.00882': [['Background'],
  ['Background', 'Limitation'],
  ['Background', 'Purpose'],
  ['Method', 'Result'],
  ['Method', 'Result'],
  ['Result', 'Conclusions'],
  ['Result', 'Conclusions']],
 '1408.1268': [['Background', 'Purpose', 'Method', 'Conclusions'],
  ['Others'],
  ['Method'],
  ['Result'],
  ['Limitation'],
  ['Result'],
  ['Result']],
 '1701.03364': [['Others'],
  ['Others'],
  ['Others'],
  ['Others'],
  ['Others'],
  ['Others'],
  ['Others']],
 '1001.1013': [['Background', 'Purpose'],
  ['Method'],
  ['Method'],
  ['Method'],
  ['Method'],
  ['Result'],
  ['Result'],
  ['Conclusions']],
 '1610.02979': [['Background', 'Purpose'],
  ['Method', 'Result', 'Conclusions']],
 '1601.0577': [['Background'],
  ['Purpose', 'Limitation'],
  ['M

In [102]:
# save to json file
import json
with open('cs_abstracts.json', 'w') as fp:
    json.dump(abstracts, fp)

with open('cs_facets.json', 'w') as fp:
    json.dump(facets_for_abstracts, fp)

In [101]:
len(facets_for_abstracts)

500